In [11]:
import torch
import torchvision
from torchvision import datasets, transforms, models

In [12]:
transform_cnn = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

transform_vit = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])


In [13]:


trainset_vit = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform_vit)

testset_vit = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform_vit)

trainset = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform_cnn)

testset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform_cnn)

trainloader_vit = torch.utils.data.DataLoader(
    trainset_vit, batch_size=128, shuffle=True)

testloader_vit = torch.utils.data.DataLoader(
    testset_vit, batch_size=128, shuffle=False)

trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=128, shuffle=True)

testloader = torch.utils.data.DataLoader(
    testset, batch_size=128, shuffle=False)

In [14]:
import torch.nn as nn
import torch.nn.functional as F

class SimpleCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool = nn.MaxPool2d(2,2)
        self.fc1 = nn.Linear(64*8*8, 256)
        self.fc2 = nn.Linear(256, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        return self.fc2(x)


In [15]:
from torchvision.models import vit_b_16

model_vit = vit_b_16(weights="IMAGENET1K_V1")
model_vit.heads.head = nn.Linear(
    model_vit.heads.head.in_features, 10)

# freeze backbone
for param in model_vit.parameters():
    param.requires_grad = False

# train ΜΟΝΟ το head
for param in model_vit.heads.head.parameters():
    param.requires_grad = True


In [16]:
from tqdm import tqdm


def train_and_test(model, train_loader, test_loader,
                   epochs, lr, name):
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model.to(device)

    criterion = nn.CrossEntropyLoss()
    if hasattr(model, "heads"):  # για ViT
        optimizer = torch.optim.Adam(model.heads.head.parameters(), lr=lr)
    else:  # για CNN
        optimizer = torch.optim.Adam(model.parameters(), lr=lr)


    print(f"\nTraining {name}")

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0

        pbar = tqdm(train_loader,
                    desc=f"Epoch [{epoch+1}/{epochs}]")

        for images, labels in pbar:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            pbar.set_postfix(loss=loss.item())

        print(f"Epoch [{epoch+1}/{epochs}] - "
              f"Loss: {running_loss/len(train_loader):.4f}")

    # ---- TEST ----
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    acc = 100 * correct / total
    print(f"{name} Accuracy: {acc:.2f}%")
    return acc


In [17]:
cnn = SimpleCNN()

acc_cnn = train_and_test(
    cnn,
    trainloader,
    testloader,
    epochs=5,
    lr=1e-3,
    name="CNN (5 epochs)"
)



Training CNN (5 epochs)


Epoch [1/5]: 100%|██████████| 391/391 [00:14<00:00, 26.83it/s, loss=1.23] 


Epoch [1/5] - Loss: 1.3963


Epoch [2/5]: 100%|██████████| 391/391 [00:14<00:00, 27.59it/s, loss=0.748]


Epoch [2/5] - Loss: 1.0092


...

Epoch [3/5] - Loss: 0.8578


Epoch [4/5]: 100%|██████████| 391/391 [00:14<00:00, 27.54it/s, loss=0.603]


Epoch [4/5] - Loss: 0.7464


Epoch [5/5]: 100%|██████████| 391/391 [00:14<00:00, 27.56it/s, loss=0.717]


Epoch [5/5] - Loss: 0.6491
CNN (5 epochs) Accuracy: 72.24%


In [18]:
acc3 = train_and_test(
    model_vit,
    train_loader=trainloader_vit,
    test_loader=testloader_vit,
    epochs=5,
    lr=1e-3,
    name="ViT (5 epochs)"
)



Training ViT (5 epochs)


Epoch [1/5]: 100%|██████████| 391/391 [16:57<00:00,  2.60s/it, loss=0.235]


Epoch [1/5] - Loss: 0.3072


Epoch [2/5]: 100%|██████████| 391/391 [16:56<00:00,  2.60s/it, loss=0.153] 


Epoch [2/5] - Loss: 0.1764


...

Epoch [3/5] - Loss: 0.1566


Epoch [4/5]: 100%|██████████| 391/391 [16:56<00:00,  2.60s/it, loss=0.114] 


Epoch [4/5] - Loss: 0.1445


Epoch [5/5]: 100%|██████████| 391/391 [16:51<00:00,  2.59s/it, loss=0.143] 


Epoch [5/5] - Loss: 0.1367
ViT (5 epochs) Accuracy: 94.49%


In [19]:
print("\nSummary:")
print(f"CNN 5 epochs: {acc_cnn:.2f}%")
print(f"ViT 5 epochs: {acc3:.2f}%")



Summary:


NameError: name 'acc1' is not defined